<a href="https://colab.research.google.com/github/JapiKredi/Pinnacle_AI_program_AnalyticsVidyha/blob/main/Assignment_nyc_taxi_trip_duration_JasperBongers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Importing the dependencies
import numpy as np
import pandas as pd
import datetime as dt
from time import strftime
from geopy.distance import geodesic
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from IPython.display import clear_output

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Reading the dataset
data = pd.read_csv('/content/drive/My Drive/Assignment_nyc_taxi_trip_duration_JasperBongers/nyc_taxi_trip_duration.csv')

In [6]:
# Head of the data
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,N,400
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,N,1100
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,N,1635
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,N,1141
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,N,848


In [7]:
# Tail of the data
data.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
729317,id3905982,2,2016-05-21 13:29:38,2016-05-21 13:34:34,2,-73.965919,40.789780,-73.952637,40.789181,N,296
729318,id0102861,1,2016-02-22 00:43:11,2016-02-22 00:48:26,1,-73.996666,40.737434,-74.001320,40.731911,N,315
729319,id0439699,1,2016-04-15 18:56:48,2016-04-15 19:08:01,1,-73.997849,40.761696,-74.001488,40.741207,N,673
729320,id2078912,1,2016-06-19 09:50:47,2016-06-19 09:58:14,1,-74.006706,40.708244,-74.013550,40.713814,N,447
729321,id1053441,2,2016-01-01 17:24:16,2016-01-01 17:44:40,4,-74.003342,40.743839,-73.945847,40.712841,N,1224


In [8]:
# Analysis of the shape f the dataframe
data.shape
# The dataset consists of 729322 rows and 11 columns
# 729322 taxi trips and 11 features

(729322, 11)

In [9]:
# Analysis of the datatypes of the columns
data.dtypes

,0
id,object
vendor_id,int64
pickup_datetime,object
dropoff_datetime,object
passenger_count,int64
pickup_longitude,float64
pickup_latitude,float64
dropoff_longitude,float64
dropoff_latitude,float64
store_and_fwd_flag,object


In [10]:
# Geeting the info function:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  729322 non-null  object 
 1   vendor_id           729322 non-null  int64  
 2   pickup_datetime     729322 non-null  object 
 3   dropoff_datetime    729322 non-null  object 
 4   passenger_count     729322 non-null  int64  
 5   pickup_longitude    729322 non-null  float64
 6   pickup_latitude     729322 non-null  float64
 7   dropoff_longitude   729322 non-null  float64
 8   dropoff_latitude    729322 non-null  float64
 9   store_and_fwd_flag  729322 non-null  object 
 10  trip_duration       729322 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 61.2+ MB


In [11]:
# Check for missing values
data.isnull().sum()
# Conclusion: no missing values

,0
id,0
vendor_id,0
pickup_datetime,0
dropoff_datetime,0
passenger_count,0
pickup_longitude,0
pickup_latitude,0
dropoff_longitude,0
dropoff_latitude,0
store_and_fwd_flag,0


In [12]:
# Lets see the dataset description (for all numerical columns)
data.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,729322.000000,729322.000000,729322.000000,729322.000000,729322.000000,729322.000000,7.293220e+05
mean,1.535403,1.662055,-73.973513,40.750919,-73.973422,40.751775,9.522291e+02
std,0.498745,1.312446,0.069754,0.033594,0.069588,0.036037,3.864626e+03
min,1.000000,0.000000,-121.933342,34.712234,-121.933304,32.181141,1.000000e+00
25%,1.000000,1.000000,-73.991859,40.737335,-73.991318,40.735931,3.970000e+02
50%,2.000000,1.000000,-73.981758,40.754070,-73.979759,40.754509,6.630000e+02
75%,2.000000,2.000000,-73.967361,40.768314,-73.963036,40.769741,1.075000e+03
max,2.000000,9.000000,-65.897385,51.881084,-65.897385,43.921028,1.939736e+06


In [13]:
## cheaking if any duplicate values
data.duplicated().sum()
## no duplicated values

0

# Encoding

let's analyze each column in the provided data and suggest appropriate encoding techniques based on the nature of the data:

1. **id**: a unique identifier for each trip.

2. **Vendor_id**: a code indicating the provider associated with the trip record.

3. **Pickup_datetime**: date and time when the meter was engaged.

4. **Dropoff_datetime**: date and time when the meter was disengaged.

5. **Passenger_count**: the number of passengers in the vehicle (driver entered value).

6. **Pickup_longitude**: the longitude where the meter was engaged.

7. **pickup_latitude**: the latitude where the meter was engaged.

8. **Dropoff_longitude**: the longitude where the meter was disengaged.

9. **Dropoff_latitude**: the latitude where the meter was disengaged.

10. **Store_and_fwd _flag**: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server (Y=store and forward; N=not a store and forward trip).

11. **trip_duration**: (target) duration of the trip in seconds.


# Integer Data Type


In [14]:
# Identifying variables with integer datatype
data.dtypes[data.dtypes == 'int64']

,0
vendor_id,int64
passenger_count,int64
trip_duration,int64


**vendor_id** -> it is okay for this to be an int64.

**passenger_count** -> very good for this to be an int64.

**trip_duration** -> number of seconds can be best translated into a datetime object. If we want to understand the duration of events in a more human-readable format (e.g., hours, minutes, seconds), converting the seconds into a timedelta object can be helpful.


In [15]:
# Assuming 'data' is your DataFrame and 'event_duration' is the column with the number of seconds
data['trip_duration'] = pd.to_timedelta(data['trip_duration'], unit='s')

In [16]:
# Verify if the trip_duration feature was changed into a Datetime object
data.info()
# Conclusion: Yes! properly converted.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype          
---  ------              --------------   -----          
 0   id                  729322 non-null  object         
 1   vendor_id           729322 non-null  int64          
 2   pickup_datetime     729322 non-null  object         
 3   dropoff_datetime    729322 non-null  object         
 4   passenger_count     729322 non-null  int64          
 5   pickup_longitude    729322 non-null  float64        
 6   pickup_latitude     729322 non-null  float64        
 7   dropoff_longitude   729322 non-null  float64        
 8   dropoff_latitude    729322 non-null  float64        
 9   store_and_fwd_flag  729322 non-null  object         
 10  trip_duration       729322 non-null  timedelta64[ns]
dtypes: float64(4), int64(2), object(4), timedelta64[ns](1)
memory usage: 61.2+ MB


# Float Data Type

In [17]:
# Identifying variables with float datatype
data.dtypes[data.dtypes == 'float64']

,0
pickup_longitude,float64
pickup_latitude,float64
dropoff_longitude,float64
dropoff_latitude,float64


Longitude and latitude values require precision, which is well-supported by the float64 data type.

So no need to convert these.

# Object Data Type

In [18]:
# Identifying variables with float datatype
data.dtypes[data.dtypes == 'object']

,0
id,object
pickup_datetime,object
dropoff_datetime,object
store_and_fwd_flag,object


**pickup_datetime** -> good to convert into a datetime object.

**dropoff_datetime** -> good to convert into a datetime object.

**store_and_fwd_flag** -> good to convert this into 0's and 1's.



In [19]:
# Convert pickup_datetime and dropoff_datetime into a Datetime object
data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'])
data['dropoff_datetime'] = pd.to_datetime(data['dropoff_datetime'])

In [20]:
# Checking if the datatype was converted properly
data.info()
# converted from Object to float64. Need to convert this into int64.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype          
---  ------              --------------   -----          
 0   id                  729322 non-null  object         
 1   vendor_id           729322 non-null  int64          
 2   pickup_datetime     729322 non-null  datetime64[ns] 
 3   dropoff_datetime    729322 non-null  datetime64[ns] 
 4   passenger_count     729322 non-null  int64          
 5   pickup_longitude    729322 non-null  float64        
 6   pickup_latitude     729322 non-null  float64        
 7   dropoff_longitude   729322 non-null  float64        
 8   dropoff_latitude    729322 non-null  float64        
 9   store_and_fwd_flag  729322 non-null  object         
 10  trip_duration       729322 non-null  timedelta64[ns]
dtypes: datetime64[ns](2), float64(4), int64(2), object(2), timedelta64[ns](1)
memory usage: 61.2+ MB


In [21]:
# Use pd.get_dummies to convert 'store_and_fwd_flag' into numerical values
data = pd.get_dummies(data, columns=['store_and_fwd_flag'])

In [22]:
# Checking if the datatype was converted properly
data.info()
# converted from Object to float64. Need to convert this into int64.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype          
---  ------                --------------   -----          
 0   id                    729322 non-null  object         
 1   vendor_id             729322 non-null  int64          
 2   pickup_datetime       729322 non-null  datetime64[ns] 
 3   dropoff_datetime      729322 non-null  datetime64[ns] 
 4   passenger_count       729322 non-null  int64          
 5   pickup_longitude      729322 non-null  float64        
 6   pickup_latitude       729322 non-null  float64        
 7   dropoff_longitude     729322 non-null  float64        
 8   dropoff_latitude      729322 non-null  float64        
 9   trip_duration         729322 non-null  timedelta64[ns]
 10  store_and_fwd_flag_N  729322 non-null  bool           
 11  store_and_fwd_flag_Y  729322 non-null  bool           
dtypes: bool(2), datetime64[ns](2), float64(4), i

In [23]:
# Convert the 'store_and_fwd_flag' column to int64
data['store_and_fwd_flag_N'] = data['store_and_fwd_flag_N'].astype('int64')

In [26]:
# Checking the unique values of store_and_fwd_flag_N
print(data['store_and_fwd_flag_N'].unique())

[1 0]


In [27]:
# Convert the 'store_and_fwd_flag' column to int64
data['store_and_fwd_flag_Y'] = data['store_and_fwd_flag_Y'].astype('int64')

In [28]:
# Checking if the datatype was converted properly
data.info()
# Successfully converted to int64.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype          
---  ------                --------------   -----          
 0   id                    729322 non-null  object         
 1   vendor_id             729322 non-null  int64          
 2   pickup_datetime       729322 non-null  datetime64[ns] 
 3   dropoff_datetime      729322 non-null  datetime64[ns] 
 4   passenger_count       729322 non-null  int64          
 5   pickup_longitude      729322 non-null  float64        
 6   pickup_latitude       729322 non-null  float64        
 7   dropoff_longitude     729322 non-null  float64        
 8   dropoff_latitude      729322 non-null  float64        
 9   trip_duration         729322 non-null  timedelta64[ns]
 10  store_and_fwd_flag_N  729322 non-null  int64          
 11  store_and_fwd_flag_Y  729322 non-null  int64          
 12  weekday               729322 non-null  objec

In [29]:
# Checking the unique values of store_and_fwd_flag_Y
print(data['store_and_fwd_flag_Y'].unique())

[0 1]


### Datetimeobject

We can craate new Datetime objects by some simple calculations on Datetime objects


In [30]:
# Create a new variable real_trip_duration by taking the difference between pickup_datetime and dropoff_datetime
data['real_trip_duration'] = data['dropoff_datetime'] - data['pickup_datetime']

In [31]:
# Get the head function for dataframe data
data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,store_and_fwd_flag_N,store_and_fwd_flag_Y,weekday,month,weekday_num,pickup_hour,real_trip_duration
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,0 days 00:06:40,1,0,Monday,2,0,16,0 days 00:06:40
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,0 days 00:18:20,1,0,Friday,3,4,23,0 days 00:18:20
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,0 days 00:27:15,1,0,Sunday,2,6,17,0 days 00:27:15
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,0 days 00:19:01,1,0,Tuesday,1,1,9,0 days 00:19:01
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,0 days 00:14:08,1,0,Wednesday,2,2,6,0 days 00:14:08


In [32]:
# Comparing (target) trip_duration with real_trip_duration
data[['trip_duration', 'real_trip_duration']].sum()
# The (target) trip_duration with real_trip_duration are excately the same

,0
trip_duration,8037 days 23:34:16
real_trip_duration,8037 days 23:34:16


In [33]:
# Check if the columns are exactly the same
are_equal = data['trip_duration'].equals(data['real_trip_duration'])
print(f"The columns are exactly the same: {are_equal}")
# The (target) trip_duration with real_trip_duration are excately the same

The columns are exactly the same: True


In [34]:
# As real_trip_duration is excately the sale as trip_duration
# remove the column real_trip_duration from the dataset as it does not contain any new information.
data.drop('real_trip_duration', axis=1, inplace=True)

In [35]:
# Change trip_duration from a Datatime datatype to int64 in order to make the processing faster and consume less memory
data['trip_duration'] = data['trip_duration'].astype('int64')

In [36]:
# Check if both changes have been implemented properly
data.info()
# Conclusion: Yes!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    729322 non-null  object        
 1   vendor_id             729322 non-null  int64         
 2   pickup_datetime       729322 non-null  datetime64[ns]
 3   dropoff_datetime      729322 non-null  datetime64[ns]
 4   passenger_count       729322 non-null  int64         
 5   pickup_longitude      729322 non-null  float64       
 6   pickup_latitude       729322 non-null  float64       
 7   dropoff_longitude     729322 non-null  float64       
 8   dropoff_latitude      729322 non-null  float64       
 9   trip_duration         729322 non-null  int64         
 10  store_and_fwd_flag_N  729322 non-null  int64         
 11  store_and_fwd_flag_Y  729322 non-null  int64         
 12  weekday               729322 non-null  object        
 13 

#### Create new features with the Datetime features


In [25]:
# Calculate and assign new columns to the dataframe such as weekday,
# Month and pickup_hour which will help us to gain more insights from the data.
data['weekday'] = data.pickup_datetime.dt.day_name() # Changed weekday_name to day_name()
data['month'] = data.pickup_datetime.dt.month
data['weekday_num'] = data.pickup_datetime.dt.weekday
data['pickup_hour'] = data.pickup_datetime.dt.hour

## Calculate distance between pickup and dropoff coordinates using geodesic.

In [39]:
# Import necessary libraries
from geopy.distance import geodesic
from IPython.display import clear_output
import pandas as pd
from tqdm import tqdm

# Initialize an empty list to store distances
distance = []

# Iterate over each index in the 'pickup_latitude' column of the DataFrame
for index in tqdm(data['pickup_latitude'].index):
    # Clear the output in the console
    clear_output(wait=True)

    # Calculate the geodesic distance between pickup and dropoff coordinates
    # and append the distance in miles to the 'distance' list
    distance.append(geodesic(
        (data['pickup_latitude'].iloc[index], data['pickup_longitude'].iloc[index]),
        (data['dropoff_latitude'].iloc[index], data['dropoff_longitude'].iloc[index])
    ).miles)

# Add the calculated distances as a new column 'distance' in the DataFrame
data['distance'] = distance

100%|██████████| 729322/729322 [34:30<00:00, 352.26it/s]


In [40]:
# Checking dataframe data if the distance feature has been created properly
data.head()
# Conclusion: Yes, the distance feature looks good to go

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,store_and_fwd_flag_N,store_and_fwd_flag_Y,weekday,month,weekday_num,pickup_hour,distance
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,400000000000,1,0,Monday,2,0,16,0.745502
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,1100000000000,1,0,Friday,3,4,23,2.562501
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,1635000000000,1,0,Sunday,2,6,17,4.505212
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,1141000000000,1,0,Tuesday,1,1,9,1.465371
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,848000000000,1,0,Wednesday,2,2,6,2.689391


In [42]:
# Checking dataframe data if the distance feature has been created properly
data.info()
# Conclusion: Yes, the distance feature looks good to go

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    729322 non-null  object        
 1   vendor_id             729322 non-null  int64         
 2   pickup_datetime       729322 non-null  datetime64[ns]
 3   dropoff_datetime      729322 non-null  datetime64[ns]
 4   passenger_count       729322 non-null  int64         
 5   pickup_longitude      729322 non-null  float64       
 6   pickup_latitude       729322 non-null  float64       
 7   dropoff_longitude     729322 non-null  float64       
 8   dropoff_latitude      729322 non-null  float64       
 9   trip_duration         729322 non-null  int64         
 10  store_and_fwd_flag_N  729322 non-null  int64         
 11  store_and_fwd_flag_Y  729322 non-null  int64         
 12  weekday               729322 non-null  object        
 13 

### Creating new Feature: Calculate Speed in miles/hr for further insights

In [41]:
data['speed'] = (data.distance/(data.trip_duration/3600))

In [43]:
# Checking dataframe data if the speed feature has been created properly
data.head()
# Conclusion: Yes, the speed feature looks good to go

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration,store_and_fwd_flag_N,store_and_fwd_flag_Y,weekday,month,weekday_num,pickup_hour,distance,speed
0,id1080784,2,2016-02-29 16:40:21,2016-02-29 16:47:01,1,-73.953918,40.778873,-73.963875,40.771164,400000000000,1,0,Monday,2,0,16,0.745502,6.709522e-09
1,id0889885,1,2016-03-11 23:35:37,2016-03-11 23:53:57,2,-73.988312,40.731743,-73.994751,40.694931,1100000000000,1,0,Friday,3,4,23,2.562501,8.386366e-09
2,id0857912,2,2016-02-21 17:59:33,2016-02-21 18:26:48,2,-73.997314,40.721458,-73.948029,40.774918,1635000000000,1,0,Sunday,2,6,17,4.505212,9.919733e-09
3,id3744273,2,2016-01-05 09:44:31,2016-01-05 10:03:32,6,-73.961670,40.759720,-73.956779,40.780628,1141000000000,1,0,Tuesday,1,1,9,1.465371,4.623433e-09
4,id0232939,1,2016-02-17 06:42:23,2016-02-17 06:56:31,1,-74.017120,40.708469,-73.988182,40.740631,848000000000,1,0,Wednesday,2,2,6,2.689391,1.141723e-08


In [44]:
# Checking dataframe data if the speed feature has been created properly
data.info()
# Conclusion: Yes, the speed feature looks good to go

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 729322 entries, 0 to 729321
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   id                    729322 non-null  object        
 1   vendor_id             729322 non-null  int64         
 2   pickup_datetime       729322 non-null  datetime64[ns]
 3   dropoff_datetime      729322 non-null  datetime64[ns]
 4   passenger_count       729322 non-null  int64         
 5   pickup_longitude      729322 non-null  float64       
 6   pickup_latitude       729322 non-null  float64       
 7   dropoff_longitude     729322 non-null  float64       
 8   dropoff_latitude      729322 non-null  float64       
 9   trip_duration         729322 non-null  int64         
 10  store_and_fwd_flag_N  729322 non-null  int64         
 11  store_and_fwd_flag_Y  729322 non-null  int64         
 12  weekday               729322 non-null  object        
 13 

## Updated list of features

let's analyze each column in the provided data and suggest appropriate encoding techniques based on the nature of the data:

1. **id**: a unique identifier for each trip.

2. **Vendor_id**: a code indicating the provider associated with the trip record.

3. **Pickup_datetime**: date and time when the meter was engaged.

4. **Dropoff_datetime**: date and time when the meter was disengaged.

5. **Passenger_count**: the number of passengers in the vehicle (driver entered value).

6. **Pickup_longitude**: the longitude where the meter was engaged.

7. **pickup_latitude**: the latitude where the meter was engaged.

8. **Dropoff_longitude**: the longitude where the meter was disengaged.

9. **Dropoff_latitude**: the latitude where the meter was disengaged.

10. **trip_duration**: (target) duration of the trip in seconds.

11. **store_and_fwd_flag_N**: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server (Y=store and forward; N=not a store and forward trip).

12. **store_and_fwd_flag_Y**: This flag indicates whether the trip record was held in vehicle memory before sending to the vendor because the vehicle did not have a connection to the server (Y=store and forward; N=not a store and forward trip).

13. **weekday**: We used the datetime object to get the day of the week of the trip.

14. **month**: We used the datetime object to get the month of the trip.

15. **weekday_num**: We used the datetime object to get the weekday number of the trip.

16. **pickup_hour**: We used the datetime object to get the hour of the pickup of the trip.

17. **distance**: We have used the geospecial data (longitude and lattitude) to calculate the distance between PickUp Location and the DropOff Location..

18. **speed**: We have calculated the average speed as the distance dvided by the trip duration.


# Univariate analysis


Univariate analysis is the analysis of one variable. It's major purpose is to describe patterns in the data consisting of single variable.

## 1. Vender Id

## Distribution

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Taking a subset to minimize impact on memory
# Taking a subset of 10,000 rows will app generate the same distribution as the full dataset

data_subset = data.iloc[:10000]  # Take the first 10,000 rows

# Assuming 'data' is your DataFrame
plt.figure(figsize=(15, 30))

# Iterate through each column and plot on a separate subplot
for i, column in enumerate(data_subset.columns):
    plt.subplot(7, 3, i + 1)
    # Limit the number of bins to avoid excessive memory usage
    try:
        plt.title(f'Distribution of {column}')
        sns.histplot(data_subset[column], kde=True, bins=30)
        plt.xticks(rotation=45)
    except MemoryError:
        print(f"MemoryError: Skipping column {column} due to memory constraints")

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Normalized Distribution (Gaussian)

In [ ]:
# Select only the numercial data
numerical_data = data.select_dtypes(include=['int64', 'float64'])
# Remove id , vendor_id, store_and_fwd_flag_N, store_and_fwd_flag_Y, pickup_longitude, pickup_latitude, dropoff_longitude, dropoff_latitude
numerical_data = numerical_data.drop(['vendor_id', 'store_and_fwd_flag_N', 'store_and_fwd_flag_Y', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude'], axis=1)

In [ ]:
# Taking a subset to minimize impact on memory
# Taking a subset of 10,000 rows will app generate the same distribution as the full dataset
data_subset = numerical_data.iloc[:10000]  # Take the first 10,000 rows

from scipy import stats

plt.figure(figsize=(15, 30))

# Iterate through each column and plot on a separate subplot
for i, column in enumerate(data_subset.columns):
    plt.subplot(7, 3, i+1)
    sns.histplot(data_subset[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.xticks(rotation=45)

    # Add additional analysis to detect distribution type
    # Check for normal distribution
    k2, p = stats.normaltest(data_subset[column])
    if p < 0.05:
        plt.text(0.5, 0.5, "Not Normal", horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)
    else:
        plt.text(0.5, 0.5, "Normal", horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Skewed Distribution (negative and positive)


In [ ]:
# Taking a subset to minimize impact on memory
# Taking a subset of 10,000 rows will app generate the same distribution as the full dataset
data_subset = numerical_data.iloc[:10000]  # Take the first 10,000 rows

# Taking a subset to minimize impact on memory
plt.figure(figsize=(15, 10))

# Iterate through each column and plot on a separate subplot
for i, column in enumerate(data_subset.columns):
    plt.subplot(7, 3, i+1)
    sns.histplot(data_subset[column], kde=True)
    plt.title(f'Distribution of {column}')
    plt.xticks(rotation=45)

    # Check for skewness
    skewness = stats.skew(data[column])
    if skewness < -1 or skewness > 1:
        plt.text(0.5, 0.3, f"Skewed ({skewness:.2f})", horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)
    else:
        plt.text(0.5, 0.3, f"Not Skewed", horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes)

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

## Outliers


In [ ]:
# For the outlier analysis we cannot take a subset.

plt.figure(figsize=(15, 30))

# Iterate through each column and plot on a separate subplot
for i, column in enumerate(numerical_data.columns):
    plt.subplot(7, 3, i+1)
    sns.boxplot(numerical_data[column])
    plt.title(f'Box plot of {column}')

# Adjust layout and show the plot
plt.tight_layout()
plt.show()

# Bivariate analysis

In [ ]:
# Calculate the correlation matrix
correlation_matrix = numerical_data.corr()

# Print the correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Heatmap')
plt.show()

#Data Preprocessing

### PLanning:
1) Remove redudant columns

2) Creating new derrived features

3) Removing outliers

4) Scaling the data




# 1) Removing redundant features

In [ ]:
data = data.drop(['id','vendor_id', ],axis = 1)

#2) Creating new derrived features

### 1) Creating a new distance feature by comparing the longitude and latitude features.
### 2) Creating new features of the datetime to capture the moment of thye week and to moment in the day


## 1) Creating a new distance feature

In [ ]:
data.head()

In [ ]:
# Use the formula of pytharas to calculate the distance travelled via SquareRooth((dropoff_longitude-pickup_longitude)^2 + (dropoff_latitude-pickup_latitude)^2)
data['distance'] = np.sqrt((data['dropoff_longitude'] - data['pickup_longitude'])**2 + (data['dropoff_latitude'] - data['pickup_latitude'])**2)

In [ ]:
data.head()

In [ ]:
data.info()

## 2) Creating new features of the datetime to capture the moment of thye week and to moment in the day


'pickup_datetime','dropoff_datetime'

In [ ]:
data.head()

In [ ]:
# Creating new features out of the pickup_datetime, like day of the week, day in the month, time at the day
data['pickup_day_of_week'] = data['pickup_datetime'].dt.day_name()
data['pickup_day_of_month'] = data['pickup_datetime'].dt.day
data['pickup_time'] = data['pickup_datetime'].dt.time

In [ ]:
data.head()

In [ ]:
data.info()

#Removing Outliers

In [ ]:
data.describe()

# Conclusions w.r.t. Outliers

We see potential outliers at the following features...

1) trip_duration -> 1.939736e+15 seconds is a lot

2) distance -> there are some very high numbers


#### Handling outliers @ trip_duration

In [ ]:
data.trip_duration.describe()

In [ ]:
data['trip_duration'].quantile(0.25), data['distance'].quantile(0.75)

In [ ]:
IQR = data['trip_duration'].quantile(0.75) - data['trip_duration'].quantile(0.25)
IQR

In [ ]:
Q1 = data['trip_duration'].quantile(0.25)
Q3 = data['trip_duration'].quantile(0.75)

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

#### Handling outliers @ distance

In [ ]:
data.distance.describe()

In [ ]:
data['distance'].quantile(0.25), data['distance'].quantile(0.75)

In [ ]:
IQR = data['distance'].quantile(0.75) - data['distance'].quantile(0.25)
IQR

In [ ]:
Q1 = data['distance'].quantile(0.25)
Q3 = data['distance'].quantile(0.75)

whisker_1 = Q1 - (1.5*IQR)
whisker_2 = Q3 + (1.5*IQR)

whisker_1, whisker_2

In [ ]:
df_df_og = df.copy()
# Drop the columns specified in the num_cols list from the DataFrame df.
# The axis=1 parameter indicates that columns are being dropped (as opposed to rows).
df = df.drop(columns = num_cols,axis = 1)
# Merging the Standardized Columns Back into the DataFrame
# The how="left" parameter specifies a left join,
# meaning all rows from the original DataFrame df will be retained,
# and the corresponding rows from the scaled DataFrame will be added.
df = df.merge(scaled,left_index=True,right_index=True,how = "left")

# Isolating the dependent (output/target) variable

In [ ]:
y_all = df.churn
df = df.drop(['churn','customer_id','occupation'],axis = 1)

## Model Building and Evaluation Metrics
Since this is a binary classification problem, we could use the following 2 popular metrics:

1. Recall
2. Area under the Receiver operating characteristic curve

Now, we are looking at the recall value here because a customer falsely marked as churn would not be as bad as a customer who was not detected as a churning customer and appropriate measures were not taken by the bank to stop him/her from churning

The ROC AUC is the area under the curve when plotting the (normalized) true positive rate (x-axis) and the false positive rate (y-axis).

Our main metric here would be Recall values, while AUC ROC Score would take care of how well predicted probabilites are able to differentiate between the 2 classes.

### Baseline Columns

In [ ]:
df.head()

In [ ]:
y_all.head()

In [ ]:
baseline_cols = ['current_month_debit', 'previous_month_debit','current_balance','previous_month_end_balance','vintage'
                 ,'occupation_retired', 'occupation_salaried','occupation_self_employed', 'occupation_student']

In [ ]:
df_baseline = df[baseline_cols]

### Train Test Split to create a validation set

In [ ]:
# Splitting the data into Train and Validation set
xtrain, xtest, ytrain, ytest = train_test_split(df_baseline,y_all,test_size=1/3, random_state=11, stratify = y_all)

In [ ]:
model = LogisticRegression()
model.fit(xtrain,ytrain)
pred = model.predict_proba(xtest)[:,1]

### AUC ROC Curve & Confusion Matrix

Now, let us quickly look at the AUC-ROC curve for our logistic regression model and also the confusion matrix to see where the logistic regression model is failing here.

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, _ = roc_curve(ytest,pred)
auc = roc_auc_score(ytest, pred)
plt.figure(figsize=(12,8))
plt.plot(fpr,tpr,label="Validation AUC-ROC="+str(auc))
x = np.linspace(0, 1, 1000)
plt.plot(x, x, linestyle='-')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc=4)
plt.show()

In [ ]:
# Confusion Matrix
pred_val = model.predict(xtest)

In [ ]:
label_preds = pred_val

cm = confusion_matrix(ytest,label_preds)


def plot_confusion_matrix(cm, normalized=True, cmap='bone'):
    plt.figure(figsize=[7, 6])
    norm_cm = cm
    if normalized:
        norm_cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        sns.heatmap(norm_cm, annot=cm, fmt='g', xticklabels=['Predicted: No','Predicted: Yes'], yticklabels=['Actual: No','Actual: Yes'], cmap=cmap)

plot_confusion_matrix(cm, ['No', 'Yes'])

In [ ]:
# Recall Score
recall_score(ytest,pred_val)